In [ ]:
from component import tile
from component import model

In [ ]:
tb_model = model.TableModel()

In [ ]:
# tiles to display the points on a map
tb_map_tile = tile.MapTile()

In [ ]:
# output to display messages
file_tile = tile.FileTile(tb_model, tb_map_tile.map)

In [ ]:
# create a test downloader
test_tile = tile.TestTile(file_tile)

In [ ]:
test_tile

In [ ]:
file_tile

In [ ]:
tb_map_tile